In [1]:
! pip3 install python-dotenv

In [9]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [ ]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "김지훈"
os.environ["LANGCHAIN_API_KEY"] = ""

In [11]:
! pip3 install langchain_text_splitters langchain_community langchain_openai pymupdf faiss-cpu

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [13]:
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("./예제파일.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 18


In [14]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 32


In [15]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

In [16]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [17]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [18]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
retriever.invoke("본래 2015년까지 도타 2의 상금 크라우드 펀딩 시스템")

[Document(id='bd02d360-d755-4d88-a939-deeec379d472', metadata={'producer': 'Skia/PDF m131', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36', 'creationdate': '2024-11-23T02:51:56+00:00', 'source': './예제파일.pdf', 'file_path': './예제파일.pdf', 'total_pages': 18, 'format': 'PDF 1.4', 'title': '리그 오브 레전드 월드 챔피언십 - 나무위키', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-11-23T02:51:56+00:00', 'trapped': '', 'modDate': "D:20241123025156+00'00'", 'creationDate': "D:20241123025156+00'00'", 'page': 3}, page_content='이름으로 내주는 우승 스킨 시스템과 어마무시한 상금에서도 나오는데, 일단 스킨은 리그 오브 레전드라는 게임에서 우승한\n선수와 팀의 이름이 영원히 기록된다는 상징적인 의미가 있으며 \'게임에 자신의 이름으로 된 스킨\'이 나오는 것과 \'자신을 상징\n하는 챔피언\'이 생기는 것만으로도 엄청난 명예가 되기에 선수들에게 동기부여를 해주는 어마무시한 원동력이 된다.[10] 이런\n높은 위상으로 인해 진출이라도 한 번 했다는 것만으로도 선수나 팀의 위상도 크게 달라진다.\n다음으로는 상금 규모가 있는데, 본래 2015년까지 도타 2의 상금 크라우드 펀딩 시스템을 "우리는 상금을 유저들에게 구걸하\n지 않는다"[11]라는 망언까지 하며 까내리며 크라우드 펀딩을 하지 않았으나 2016 시즌부터 언제 그랬냐는 

In [19]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)

In [ ]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [21]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [30]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "월즈를 지배한 OP챔피언은 뭐가 있어?"  # 질문은 변경될 수 있음
response = chain.invoke(question)

# 출력 결과가 '모르겠어요.'인지 확인
if any(keyword in response.strip() for keyword in ["모르겠어요" , "없습니다", "제공된 문서에 포함되어 있지 않습니다"]):
    print("문서에서 답변을 찾을 수 없습니다. ChatGPT에 질문을 전달합니다...")

    # ChatGPT 모델 호출
    from langchain_openai import ChatOpenAI
    llm_chatgpt = ChatOpenAI(model_name="gpt-4o", temperature=0)

    # ChatGPT에 질문 전달
    chatgpt_response = llm_chatgpt.invoke(question)
    print(f"ChatGPT 답변: {chatgpt_response}")
else:
    # 문서에서 답변이 있는 경우
    print(f"체인 답변: {response}")

체인 답변: 월즈를 지배한 OP 챔피언으로는 시즌 1의 누누, 시즌 2의 이즈리얼, 시즌 3의 제드, 시즌 4의 알리스타, 시즌 5의 갱플랭크와 모데카이저, 시즌 6의 니달리, 시즌 7의 칼리스타 등이 있습니다.
